In [ ]:
from qibolab import Platform
import diagnostics

In [ ]:
# Create a platform; connect and configure it
platform = Platform('tiiq')
platform.connect()
platform.setup()

# create a diagnostics object
ds = diagnostics.Diagnostics(platform)

# Characterisation can be done by changing settings to qibolab/runcards/tiiq.yml and diagnostics.yml

Resonator Spectroscopy

In [ ]:

resonator_freq, avg_min_voltage, max_ro_voltage, smooth_dataset, dataset = ds.run_resonator_spectroscopy()

In [ ]:
#save resonator params
print(diagnostics.get_config_parameter("settings", "", "resonator_freq"))
diagnostics.save_config_parameter("settings", "", "resonator_freq", float(resonator_freq))

print(diagnostics.get_config_parameter("settings", "", "resonator_spectroscopy_avg_min_ro_voltage"))
diagnostics.save_config_parameter("settings", "", "resonator_spectroscopy_avg_min_ro_voltage", float(avg_min_voltage))

print(diagnostics.get_config_parameter("settings", "", "resonator_spectroscopy_max_ro_voltage"))
diagnostics.save_config_parameter("settings", "", "resonator_spectroscopy_max_ro_voltage", float(max_ro_voltage))

print(diagnostics.get_config_parameter("LO_QRM_settings", "", "frequency"))
diagnostics.save_config_parameter("LO_QRM_settings", "", "frequency", float(resonator_freq - 20_000_000))


Qubit Spectroscopy

In [ ]:
qubit_freq, min_ro_voltage, smooth_dataset, dataset = ds.run_qubit_spectroscopy()

In [ ]:
#save qubit frequency founded - update LO_QCM freq + update qubit spec min voltage
print(diagnostics.get_config_parameter("settings", "", "qubit_freq"))
diagnostics.save_config_parameter("settings", "", "qubit_freq", float(qubit_freq))

print(diagnostics.get_config_parameter("LO_QCM_settings", "", "frequency"))
diagnostics.save_config_parameter("LO_QCM_settings", "", "frequency", float(qubit_freq + 200_000_000))

print(diagnostics.get_config_parameter("settings", "", "qubit_spectroscopy_min_ro_voltage"))
diagnostics.save_config_parameter("settings", "", "qubit_spectroscopy_min_ro_voltage", float(min_ro_voltage))

Rabi Oscillations with varying pulse length

In [ ]:
dataset, pi_pulse_duration, pi_pulse_amplitude, pi_pulse_gain, rabi_oscillations_pi_pulse_min_voltage, t1 = ds.run_rabi_pulse_length()


In [ ]:
#save pi pulse params founded + pi pulse min vol + T1
print(diagnostics.get_config_parameter("settings", "", "pi_pulse_duration"))
diagnostics.save_config_parameter("settings", "", "pi_pulse_duration", int(pi_pulse_duration))

print(diagnostics.get_config_parameter("settings", "", "pi_pulse_amplitude"))
diagnostics.save_config_parameter("settings", "", "pi_pulse_amplitude", float(pi_pulse_amplitude))

print(diagnostics.get_config_parameter("settings", "", "pi_pulse_gain"))
diagnostics.save_config_parameter("settings", "", "pi_pulse_gain", float(pi_pulse_gain))

print(diagnostics.get_config_parameter("settings", "", "rabi_oscillations_pi_pulse_min_voltage"))
diagnostics.save_config_parameter("settings", "", "rabi_oscillations_pi_pulse_min_voltage", float(rabi_oscillations_pi_pulse_min_voltage))

print(diagnostics.get_config_parameter("settings", "", "T1"))
#TODO: Double check T1 obtained from Rabi. It is not correct.
#diagnostics.save_config_parameter("settings", "", "T1", float(t1))

T1

In [ ]:
t1, smooth_dataset, dataset = ds.run_t1()

Ramsey

In [ ]:
t2, smooth_dataset, dataset = ds.run_ramsey()

Spin Echo

In [ ]:
dataset = ds.run_spin_echo()

In [ ]:
#Run Spin Echo 3 Pulses
dataset = ds.run_spin_echo_3pulses()
#TODO: The result of the plot does not match with the expected result
#        it should result in state |0> and it seems to give state |1>